In [1]:
import emoji

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
emoji.EMOJI_UNICODE

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [4]:
emoji.emojize(":flexed_biceps:")

'💪'

In [5]:
emoji_dict = {"0" : ":heart:" , "1" : ":baseball:", "2" : ":smile:", 
              "3" : ":disappointed:", "4" : ":fork_and_knife:",
              "5" : ":sunglasses:"}

In [6]:
for e in emoji_dict.values():
    print(emoji.emojize(e,use_aliases=True))

❤
⚾
😄
😞
🍴
😎


In [7]:
#processing custom dataset

In [8]:
import pandas as pd
import numpy as np

In [9]:
train= pd.read_csv("train_emoji.csv",header=None)

In [10]:
test= pd.read_csv("test_emoji.csv",header=None)

In [11]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [12]:
# Let us print the sentences with emojis

In [13]:
data= train.values

In [14]:
print(data.shape)

(132, 4)


In [15]:
X_train= train[0]
Y_train=train[1]

X_test= test[0]
Y_test=test[1]

In [16]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dict[str(Y_train[i])],use_aliases=True))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤
food is life 🍴


In [17]:
#Download Glove6B50D.txt

In [18]:
f= open("glove.6B.50d.txt",encoding='utf-8')

In [19]:
embeddings_index={}
cnt=0

for line in f:
    values= line.split()
    word= values[0]
    coefs= np.asarray(values[1:],dtype='float')
    embeddings_index[word]= coefs
f.close()



In [20]:
emb_dim=embeddings_index['eat'].shape
print(emb_dim)

(50,)


In [21]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

C:\Users\manavgakhar\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\manavgakhar\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:

embeddings_train = np.zeros((X_train.shape[0], 10, 50))
embeddings_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embeddings_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        

for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embeddings_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [23]:
print(embeddings_test.shape)

(56, 10, 50)


In [24]:
print(embeddings_train.shape)

(132, 10, 50)


## Define the RNN or LSTM model

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN, LSTM, Activation
from keras.utils import to_categorical


Using TensorFlow backend.


In [26]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)
print(Y_train.shape)

(132, 5)


In [35]:
model= Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [39]:

hist= model.fit(embeddings_train,Y_train,epochs=150,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/150
105/105 [==============================] - 0s 1ms/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 2.1499 - val_accuracy: 0.5926
Epoch 2/150
105/105 [==============================] - 0s 1ms/step - loss: 0.0235 - accuracy: 0.9905 - val_loss: 2.1465 - val_accuracy: 0.5926
Epoch 3/150
105/105 [==============================] - 0s 3ms/step - loss: 0.0629 - accuracy: 0.9714 - val_loss: 2.0156 - val_accuracy: 0.5556
Epoch 4/150
105/105 [==============================] - 0s 2ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 2.0849 - val_accuracy: 0.5556
Epoch 5/150
105/105 [==============================] - 0s 2ms/step - loss: 0.0443 - accuracy: 0.9810 - val_loss: 2.2160 - val_accuracy: 0.5556
Epoch 6/150
105/105 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 2.2441 - val_accuracy: 0.5556
Epoch 7/150
105/105 [==============================] - 0s 2ms/step - loss: 0.0100 - accuracy: 1.0

In [40]:
model.evaluate(embeddings_test,Y_test)

56/56 [==============================] - 0s 261us/step


[2.455304060663496, 0.625]

In [48]:
pred= model.predict_classes(embeddings_test)

In [49]:
pred

array([4, 3, 2, 2, 2, 2, 3, 2, 4, 2, 1, 2, 0, 0, 1, 3, 2, 2, 3, 4, 3, 0,
       4, 2, 3, 3, 2, 0, 3, 2, 0, 1, 3, 2, 0, 1, 2, 4, 4, 2, 1, 0, 0, 2,
       2, 1, 2, 2, 3, 3, 3, 0, 3, 2, 3, 4], dtype=int64)

In [56]:
Y_test= test[1]

In [59]:

def print_emoji(label):
    
    return emoji.emojize(emoji_dict[str(label)], use_aliases=True)

In [60]:
for ix in range(embeddings_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print (X_test[ix], print_emoji(Y_test[ix]), print_emoji(pred[ix]))

['she', 'got', 'me', 'a', 'present'] ❤ 😄
['he', 'is', 'a', 'good', 'friend'] ❤ 😄
['I', 'am', 'upset'] ❤ 😞
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] ❤ 😄
['work', 'is', 'hard'] 😞 😄
['This', 'girl', 'is', 'messing', 'with', 'me'] 😞 ❤
['are', 'you', 'serious', 'ha', 'ha'] 😄 ❤
['work', 'is', 'horrible'] 😞 😄
['I', 'love', 'taking', 'breaks'] ❤ 😞
['you', 'brighten', 'my', 'day'] 😄 ❤
['she', 'is', 'a', 'bully'] 😞 😄
['I', 'worked', 'during', 'my', 'birthday'] 😞 😄
['enjoy', 'your', 'break'] 😄 😞
['valentine', 'day', 'is', 'near'] ❤ 😄
['I', 'will', 'go', 'dance'] 😄 ⚾
['I', 'like', 'your', 'jacket'] 😄 ❤
['what', 'is', 'your', 'favorite', 'baseball', 'game'] ⚾ 😄
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] ❤ ⚾
['I', 'want', 'to', 'joke'] 😄 😞
['You', 'totally', 'deserve', 'this', 'prize'] 😄 😞
['I', 'did', 'not', 'have', 'breakfast'] 😞 🍴
